In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms
import pretrainedmodels
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from data import data_reader

Tensor = torch.Tensor
rand_m = np.random.random

In [2]:
MODEL_NAME = 'pnasnet5large'

In [3]:
class model_pnas(nn.Module):
    def __init__(self, num_class):
        super(model_pnas, self).__init__()
        self.pnas = pretrainedmodels.pnasnet5large(num_classes=1000)
        self.argp = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(4320, num_class)
    
    def forward(self, x):
        x = self.pnas.features(x)
        x = self.argp(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [4]:
loss_func = torch.nn.CrossEntropyLoss()

In [5]:
from data import data_reader
class args:
    bs = 4
    path = 'train2000/'

p_norm = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
p_trm = transforms.Compose([
    transforms.Resize(331),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    p_norm
])

val_p_trm = transforms.Compose([
    transforms.Resize(333),
    transforms.CenterCrop(331),
    transforms.ToTensor(),
    p_norm
])
    
    
dr = data_reader(args, 'train2000/train2000.csv', trm=val_p_trm, val_trm=val_p_trm)

train_loader, test_loader = dr.get_train_loader()

In [6]:
lr_list = np.linspace(0.005, 0.05, 25)
print(lr_list)

[0.005    0.006875 0.00875  0.010625 0.0125   0.014375 0.01625  0.018125
 0.02     0.021875 0.02375  0.025625 0.0275   0.029375 0.03125  0.033125
 0.035    0.036875 0.03875  0.040625 0.0425   0.044375 0.04625  0.048125
 0.05    ]


In [7]:
train_loss_rec = []
train_acc_rec = []
test_loss_rec = []
test_acc_rec = []

num_epoch = 5

for learning_rate in lr_list:
    model = model_pnas(6).to('cuda')
    optim = torch.optim.Adam(model.parameters(), lr = learning_rate)
    for epoch in range(num_epoch):
        epoch_loss = 0
        total, correct = 0, 0
        for batch_idx, (X_train, y_train) in enumerate(train_loader):
            X_train, y_train = X_train.to('cuda'), y_train.to('cuda')
            optim.zero_grad()
            y_hat = model(X_train)
            loss = loss_func(y_hat, torch.max(y_train, 1)[1])
            loss.backward()
            _, pred = torch.max(y_hat.data, 1)
            optim.step()
            correct += (pred == torch.max(y_train, 1)[1]).sum().item()
            total += X_train.size(0)
            epoch_loss += loss.item() / len(train_loader)
            if (batch_idx % 50) == 0:
                print('LR {} | Epoch {} | Batch # {} Train Loss {:.5f} '.format(learning_rate, epoch, batch_idx, loss.item()))

        print('\nLR {} | Epoch {} | Epoch Train Loss {:.5f}'.format(learning_rate, epoch, epoch_loss))
        epoch_acc = correct / total * 100
        print('LR {} | Epoch {} | Epoch Train Acc {:.3f}%'.format(learning_rate, epoch, epoch_acc))

        with torch.no_grad():
            test_epoch_loss = 0
            test_total = 0
            test_correct = 0

            for batch_idx, (X_val, y_val) in enumerate(test_loader):
                X_val, y_val = X_val.to('cuda'), y_val.to('cuda')

                y_hat = model(X_val)
                loss = loss_func(y_hat, torch.max(y_val, 1)[1])
                _, pred = torch.max(y_hat.data, 1)
                test_total += y_val.size(0)
                test_correct += (pred == torch.max(y_val, 1)[1]).sum().item()
                test_epoch_loss += loss.item() / len(test_loader)

            test_epoch_acc = test_correct / test_total * 100

            print('Epoch {} | Epoch Val Loss {:.5f}'.format(epoch, test_epoch_loss))
            print('Epoch {} | Epoch Val Acc {:.3f}%\n'.format(epoch, test_epoch_acc))

    test_loss_rec.append(test_epoch_loss)
    test_acc_rec.append(test_epoch_acc)

    train_loss_rec.append(epoch_loss)
    train_acc_rec.append(epoch_acc)

Epoch 0 | Batch # 0 Train Loss 1.75981 
Epoch 0 | Batch # 0 Train Loss 1.75981 
Epoch 0 | Batch # 1 Train Loss 3.61903 
Epoch 0 | Batch # 2 Train Loss 58.78155 
Epoch 0 | Batch # 3 Train Loss 17.22742 
Epoch 0 | Batch # 4 Train Loss 186.78053 
Epoch 0 | Batch # 5 Train Loss 63.64571 
Epoch 0 | Batch # 6 Train Loss 53.16903 
Epoch 0 | Batch # 7 Train Loss 67.94562 
Epoch 0 | Batch # 8 Train Loss 46.63819 
Epoch 0 | Batch # 9 Train Loss 33.92497 
Epoch 0 | Batch # 10 Train Loss 21.89477 
Epoch 0 | Batch # 11 Train Loss 57.46309 
Epoch 0 | Batch # 12 Train Loss 30.29081 
Epoch 0 | Batch # 13 Train Loss 57.30238 
Epoch 0 | Batch # 14 Train Loss 6.19281 
Epoch 0 | Batch # 15 Train Loss 54.27122 
Epoch 0 | Batch # 16 Train Loss 32.60810 
Epoch 0 | Batch # 17 Train Loss 10.05855 
Epoch 0 | Batch # 18 Train Loss 15.83625 
Epoch 0 | Batch # 19 Train Loss 39.19742 
Epoch 0 | Batch # 20 Train Loss 44.71524 
Epoch 0 | Batch # 21 Train Loss 13.00549 
Epoch 0 | Batch # 22 Train Loss 9.91865 
Epoch 0

Process Process-5:
Traceback (most recent call last):
  File "/data/ysw/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/data/ysw/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/data/ysw/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/data/ysw/anaconda3/lib/python3.7/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/data/ysw/anaconda3/lib/python3.7/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/data/ysw/anaconda3/lib/python3.7/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/data/ysw/anaconda3/lib/python3.7/multiprocessing/connection.py", line 920, in wait
    ready = selector.select(timeout)
  File "/data/ysw/anaconda3/lib/

Epoch 0 | Batch # 13 Train Loss 6.40675 


KeyboardInterrupt: 

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
plt.subplot(2,1,1)
plt.plot(train_loss_rec, 'b')
plt.plot(test_loss_rec, 'r')

plt.title('loss')
plt.xlabel('Epoch')

plt.legend(['Train loss', 'Validation loss'], loc=4)

plt.subplot(2,1,2)
plt.plot(train_acc_rec, 'b')
plt.plot(test_acc_rec, 'r')

plt.title('Prediction Accuracy')
plt.xlabel('Epoch')

plt.legend(['Train prediction accuracy' ,'Validation prediction accuracy'], loc=4)
plt.show()

In [ ]:
import 